In [27]:
import json
from prettytable import PrettyTable

In [2]:
with open('tajriba.json', 'r') as data_file:
    lines = data_file.readlines()
    data = list(map(json.loads, lines))

In [3]:
# data

[{'version': 'v1.4.2',
  'sha': '4adfa6c',
  'tag': 'v1.4.2',
  'branch': 'main',
  'time': '2023-07-06T03:48:27Z'},
 {'version': 0, 'format': 'JSON', 'compression': 'NoCompression'},
 {'kind': 'Service',
  'obj': {'id': '01H71CVYQQ2Z7QSGE3KX982X7Y',
   'name': 'system',
   'createdAt': '2023-08-04T16:07:06.359858-07:00'}},
 {'kind': 'Scope',
  'obj': {'id': '01H71CVYQQ2Z7QSGE3KYWCJQ8M',
   'name': 'global',
   'kind': 'global',
   'createdAt': '2023-08-04T16:07:06.359981-07:00',
   'createdByID': '01H71CVYQQ2Z7QSGE3KX982X7Y'}},
 {'kind': 'User',
  'obj': {'id': '01H71CVYQRDPJVP7H95DQA76G6',
   'name': 'Admin',
   'username': 'admin',
   'createdAt': '2023-08-04T16:07:06.360019-07:00'}},
 {'kind': 'Scope',
  'obj': {'id': '01H71CW4B12K17EV1720G1HVZV',
   'name': None,
   'kind': 'batch',
   'createdAt': '2023-08-04T16:07:12.097601-07:00',
   'createdByID': '01H71CVYQQ2Z7QSGE3KX982X7Y'}},
 {'kind': 'Attribute',
  'obj': {'id': '01H71CW4B12K17EV1720GVSW4C',
   'createdAt': '2023-08-04T16

In [4]:
events = list(filter(lambda event: 'version' not in event, data))

In [5]:
# set(map(lambda event: event['kind'], events))

{'Attribute',
 'Group',
 'Link',
 'Participant',
 'Scope',
 'Service',
 'Session',
 'Step',
 'Transition',
 'User'}

In [45]:
def filter_by(key, value, events):
    return list(filter(lambda event: event[key] == value, events))

In [46]:
ids = {}
for event in filter_by('kind', 'Participant', events):
    ids[event['obj']['id']] = event['obj']['Identifier']
# ids

In [47]:
attr_events = list(map(lambda event: event['obj'], filter_by('kind', 'Attribute', events)))
# attr_events

In [48]:
# set(sum(map(lambda event: list(event.keys()), attr_events), []))

In [49]:
def parse_attr(event):
    cleaned_event = {}
    
    for key in ['key', 'val']:
        if key in event:
            cleaned_event[key] = event[key]
    
    if event['createdByID'] in ids:
        cleaned_event['player'] = ids[event['createdByID']]
    else:
        cleaned_event['player'] = ''
    
    return cleaned_event

In [50]:
cleaned_attr_events = list(map(parse_attr, attr_events))
# cleaned_attr_events

In [51]:
# set(map(lambda event: event['key'], cleaned_attr_events))

In [52]:
table = PrettyTable()
table.add_column('key', list(map(lambda event: event['key'], cleaned_attr_events)))
table.add_column('val', list(map(lambda event: event['val'], cleaned_attr_events)))
table.add_column('player', list(map(lambda event: event['player'], cleaned_attr_events)))
table

key,val,player
config,"{""kind"":""complete"",""config"":{""treatments"":[{""treatment"":{""name"":""Two Players"",""desc"":""Two-player Game"",""factors"":{""playerCount"":2}},""count"":1}]}}",
status,"""created""",
lobbyConfig,"{""name"":""Default shared fail"",""kind"":""shared"",""duration"":300000000000,""strategy"":""fail""}",
treatment,"{""playerCount"":2}",
batchID,"""01H71CW4B12K17EV1720G1HVZV""",
start,false,
ended,false,
initialized,true,
ran-before-01H71CW4BSEY95M1CBM0XQX0GM,true,
ran-on-01H71CW4BSEY95M1CBM2NVEV2N,true,


In [85]:
def parse_action(action):
    if action['val'] == 'null':
        return ''
    val = json.loads(action['val'])
    if val['type'] == 'move':
        return f"{action['player']} move: {val['position']}"
    if val['type'] == 'collect':
        return f"{action['player']} collect: {val['item']}"
    if val['type'] == 'drop':
        return f"{action['player']} drop: {val['item']}"

moves = list(map(parse_action,
         filter(lambda action: action['val'] != 'null',
                filter_by('key', 'action', cleaned_attr_events)
               )
        )
    )
moves

['e1 move: [5, 1]',
 'e2 move: [1, 5]',
 'e1 collect: 0',
 'e2 collect: 0',
 'e1 move: [4, 1]',
 'e2 move: [1, 4]',
 'e1 collect: 0',
 'e2 collect: 0']

In [86]:
boards = list(map(lambda board_event: json.loads(board_event['val']), filter(lambda action: True, filter_by('key', 'board', cleaned_attr_events))))
boards

[[[[1], [], [2], [0], [0], []],
  [[1], [2], [], [0], [0], [0]],
  [[2], [], [2], [0], [0], [0]],
  [[0], [0], [0], [2], [], [2]],
  [[0], [0], [0], [], [], [1]],
  [[], [0], [0], [2], [1], [1]]],
 [[[1], [], [2], [0], [0], []],
  [[1], [2], [], [0], [0], [0]],
  [[2], [], [2], [0], [0], [0]],
  [[0], [0], [0], [2], [], [2]],
  [[0], [0], [0], [], [], [1]],
  [[], [0], [0], [2], [1], [1]]],
 [[[1], [], [2], [0], [0], []],
  [[1], [2], [], [0], [0], []],
  [[2], [], [2], [0], [0], [0]],
  [[0], [0], [0], [2], [], [2]],
  [[0], [0], [0], [], [], [1]],
  [[], [], [0], [2], [1], [1]]],
 [[[1], [], [2], [0], [0], []],
  [[1], [2], [], [0], [0], []],
  [[2], [], [2], [0], [0], [0]],
  [[0], [0], [0], [2], [], [2]],
  [[0], [0], [0], [], [], [1]],
  [[], [], [0], [2], [1], [1]]],
 [[[1], [], [2], [0], [0], []],
  [[1], [2], [], [0], [], []],
  [[2], [], [2], [0], [0], [0]],
  [[0], [0], [0], [2], [], [2]],
  [[0], [], [0], [], [], [1]],
  [[], [], [0], [2], [1], [1]]],
 [[[1], [], [2], [0], [